In [ ]:
import os

from dotenv import load_dotenv
from huggingface_hub import login
import torch
import timm
from torchvision import transforms

from histolung.evaluation.evaluators import LungHist700Evaluator

In [ ]:
load_dotenv()
hugging_face_token = os.getenv('HUGGING_FACE_TOKEN')

In [ ]:
# Login to the Hugging Face hub, using your user access token that can be found here:
# https://huggingface.co/settings/tokens.
login(token=hugging_face_token)

model = timm.create_model(
    "hf-hub:bioptimus/H-optimus-0", pretrained=True, init_values=1e-5, dynamic_img_size=False
)

In [ ]:
model.to("cuda")
model.eval()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.707223, 0.578729, 0.703617), 
        std=(0.211883, 0.230117, 0.177517)
    ),
])

input = torch.rand(3, 224, 224)
input = transforms.ToPILImage()(input)

# We recommend using mixed precision for faster inference.
with torch.autocast(device_type="cuda", dtype=torch.float16):
    with torch.inference_mode():
        features = model(transform(input).unsqueeze(0).to("cuda"))

assert features.shape == (1, 1536)

In [ ]:
evaluator = LungHist700Evaluator(
    model=model,
    transform=transform,
    batch_size=32,
    num_workers=4,
    gpu_id=1,
)